In [1]:
import numpy as np
import networkx as nx
np.set_printoptions(threshold=100)

f=open('inputdef.txt') #not with read because thats probably the whole file
lines = [list(line.rstrip('\n')) for line in f]
grid = np.array(lines)

In [2]:
# this day I solved by hand basically
class Pos():
    def __init__(self,input):
        self.x,self.y = input[0],input[1]
        self.loc = input
        self.n = getneigh((self.x,self.y))
        self.value = grid[input]
        self.door,self.key = False,False
        if self.value.isupper(): self.door = True
        elif self.value.islower(): self.key = True
    def enter(self,keys):
        if self.door:
            values = [(locations[k].value).upper() for k in keys]
            if self.value in values: return self.loc
            else: return False
        else: return self.loc
    def __repr__(self):
        return f'({self.x},{self.y}) {self.value} is door {self.door} is key {self.key}'

In [31]:
locations = {tuple(i)  for i in np.argwhere(grid!='#')}

def getlocations():
    return {tuple(i):Pos(tuple(i)) for i in np.argwhere(grid!='#')}
def getneigh(i):
  res = []
  if (i[0],i[1]+1) in locations: res.append((i[0],i[1]+1))
  if (i[0],i[1]-1) in locations: res.append((i[0],i[1]-1))
  if (i[0]+1,i[1]) in locations: res.append((i[0]+1,i[1]))
  if (i[0]-1,i[1]) in locations: res.append((i[0]-1,i[1]))
  return res
locations = getlocations()
doors = [i.loc for i in locations.values() if i.door]
keys = [i.loc for i in locations.values() if i.key]

neigh = {tuple(i):getneigh(tuple(i)) for i in np.argwhere(grid!='#')}

In [356]:
counter = 0
for _ in range(200):
    for k,v in locations.items():
        if len(v.n)==1 and not v.key:
            grid[k]='#'
            counter +=1
    locations = getlocations()
print(counter)

0


In [8]:
for k,v in locations.items():
    # print(len(v.n),v.key)
    if len(v.n)==1 and v.key:
        print(v)

In [358]:
with open('inputdef.txt', 'a') as the_file:
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            the_file.write(str(grid[i,j]))
        the_file.write('\n')



In [9]:
len(locations)

1755

In [12]:

def get_potentials(keyset,current_loc): #return keys that are reachable, given a keyset and current_loc
    # print('getpotentials',keyset,current_loc)
    access = [v.enter(keyset) for v in locations.values() if v.enter(keyset) ]
    noaccess = [v.loc for v in locations.values() if not v.enter(keyset) ]

    g = nx.Graph()
    g.add_nodes_from(access)
    for a in locations: # for each location (without the doors)
        for n in locations[a].n: #for each neighbor
            g.add_edge(a,n)
    g.remove_nodes_from(noaccess)
    reachable =  nx.algorithms.descendants(g, current_loc)
    reachable_keys = {k for k in keys if k in reachable}
    relevant_keys  = reachable_keys-set(keyset)
    #potentially look if shortest path includes a key from relevant keys, since then it can be added?
    return {k: nx.shortest_path_length(g,current_loc,k) for k in relevant_keys}
    

In [16]:
paths = []
distances = []
from copy import deepcopy
def step(keyset,loc,length):
    res = get_potentials(keyset,loc)
    # print('possible keys for',keyset,loc,res)
    # print(len(res))
    if not res: 
        paths.append(keyset)
        distances.append(length)
    else:
        if len(res)==1:
            i = next(iter(res))
            return step(deepcopy(keyset)+[i],i,deepcopy(length)+[res[i]])
        else:
            return [step(deepcopy(keyset)+[k],k,deepcopy(length)+[v]) for k,v in res.items()]
current_loc = tuple(np.argwhere(grid=='@')[0])
test = step([],current_loc,[])

In [37]:
sol = 'ogbuvrmtincyafzplkhexsdwqj'
changedkeys = {locations[k].value:k for k in keys}
solcoor = [changedkeys[c] for c in sol]
print(len(solcoor))
solcoor = [current_loc]+solcoor
print(len(solcoor))


26
27


In [48]:

g = nx.Graph()
g.add_nodes_from(locations)
for a in locations: # for each location (without the doors)
    for n in locations[a].n: #for each neighbor
        g.add_edge(a,n)
sum([nx.shortest_path_length(g,x,y) for x,y in zip(solcoor,solcoor[1:])])

322

In [49]:
508+582+602+322

2014

In [47]:
current_loc = (41,41)
sol = 'fzpl'
changedkeys = {locations[k].value:k for k in keys}
solcoor = [changedkeys[c] for c in sol]
print(len(solcoor))
solcoor = [current_loc]+solcoor
print(len(solcoor))


4
5


In [ ]:
current_loc = (39,39)
sol = 'ogj'
current_loc = (39,41)
sol = 'mtinhexsdwq'
current_loc = (41,39)
sol = 'buvrcya'